In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [2]:
SEED = 42
BATCH_SIZE = 4
MODEL = "v2"

In [4]:
train = pd.read_csv('./정형데이터/train.csv')
valid = pd.read_csv('./정형데이터/valid.csv')

In [ ]:
pattern = r'(.+?)_(.+?)\.jpg'
train[['Vienna_Code','etc']] = train['fileName'].str.extract(pattern)
train = train.drop(columns=['etc'])
train

In [ ]:
pattern = r'(.+?)_(.+?)\.jpg'
valid[['Vienna_Code','etc']] = valid['fileName'].str.extract(pattern)
valid = valid.drop(columns=['etc'])
valid

In [14]:
# # 데이터셋 폴더 만들기
# if os.path.exists("D:/상표 이미지 및 텍스트/Training/data/yolo"):
#     shutil.rmtree("D:/상표 이미지 및 텍스트/Training/data/yolo")

# if not os.path.exists("D:/상표 이미지 및 텍스트/Training/data/yolo/train"):
#     os.makedirs("D:/상표 이미지 및 텍스트/Training/data/yolo/train")
    
# if not os.path.exists("'D:/상표 이미지 및 텍스트/Training/data/yolo/valid"):
#     os.makedirs("D:/상표 이미지 및 텍스트/Training/data/yolo/valid")

In [ ]:
## 데이터셋 만들기
# def make_yolo_dataset(df, type="train"):
#     for index, row in tqdm(df.iterrows(), total=len(df)):
#         image_num = row['image_id']
#         code_index = row['Code_Index']
#         vienna_code = row['Vienna_Code']
#         file_name = row['fileName']
#         image_path = os.path.join(f'D:/상표 이미지 및 텍스트/Training/[원천]상표이미지_{type}/{vienna_code}_TOTAL/{file_name}')
#         source_image = cv2.imread(image_path, cv2.IMREAD_COLOR) 
#         image_height, image_width, _ = source_image.shape
#         target_image_path = f"D:/상표 이미지 및 텍스트/Training/data/yolo/{type}/{file_name}"
#         cv2.imwrite(target_image_path, source_image)

#         target_label_txt = f"D:/상표 이미지 및 텍스트/Training/data/yolo/{type}/{vienna_code}_{image_num}.txt"      
#         with open(target_label_txt, "a") as writer:
#             x, y, width, height = ast.literal_eval(row['bbox'])
#             class_index = code_index
#             x_min, y_min = max(0, min(image_width, float(x))), max(0, min(image_height, float(y)))
#             width, height = max(0, min(image_width, abs(width))), max(0, min(image_height, abs(height)))
#             x, y = min(1, float(((x_min*2 + width) / 2) / image_width)), min(1,float(((y_min*2 + height) / 2) / image_height))
#             w, h = min(1, width / image_width), min(1, height / image_height)
#             writer.write(f"{class_index} {x} {y} {w} {h}\n")

In [ ]:
# # 데이터셋 만들기
# make_yolo_dataset(train_images_paths, train_txt_paths, "train")
# make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")

In [6]:
# 데이터 정보를 담은 yaml 생성하기(클래스 정보 포함)
classes_temp = train['Vienna_Code'].drop_duplicates().tolist() # 혹시 {0:'code'} 이런 형태
classes = {index: class_name for index, class_name in enumerate(classes_temp)}

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": "./data/yolo/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }
with open("./data/yolocustom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

In [ ]:
#model = YOLO(f"{MODEL}/train/weights/last.pt")

# 모델 불러오기
model = YOLO('yolov8n.yaml')  # YAML에서 새 모델 구축
# model = YOLO('yolov8n.pt')  # 사전 훈련된 모델 불러오기 (학습용 추천)
# model = YOLO('yolov8n.yaml').load('yolov8n-cls.pt')  # YAML로 구축하고 가중치 전송

# 모델 학습하기
results = model.train(
    data="./data/yolo/custom.yaml",
    imgsz=(320, 320),
    epochs=1,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,    
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,
    resume=True,
    optimizer="Adam",
    lr0=1e-3,
    augment=True,
    val=True,
    cache=True
    )